## Initialize

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

## Import the Data

In [3]:
dataset = pd.read_csv('iris.data', header = None)
dataset.shape

(150, 5)

In [4]:
dataset.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## Create X and Y

In [5]:
X = dataset.iloc[:, 0:4].values
Y = dataset.iloc[:, 4].values

In [6]:
X[0]

array([ 5.1,  3.5,  1.4,  0.2])

In [7]:
Y[0]

'Iris-setosa'

## Preprocess the Data

In [8]:
le_Y = LabelEncoder()

In [9]:
Y = le_Y.fit_transform(Y)
Y = Y.reshape(len(Y), 1)

In [10]:
ohe_Y = OneHotEncoder(categorical_features = [0])

In [11]:
Y = ohe_Y.fit_transform(Y).toarray()
Y[0]

array([ 1.,  0.,  0.])

In [12]:
sc_X = StandardScaler()

In [13]:
X = sc_X.fit_transform(X)
X[0]

array([-0.90068117,  1.03205722, -1.3412724 , -1.31297673])

## Create Train and Test Data

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 4)

In [15]:
X_train.shape

(120, 4)

In [16]:
X_test.shape

(30, 4)

In [17]:
Y_train.shape

(120, 3)

In [18]:
Y_test.shape

(30, 3)

## Create TF Classifier

In [19]:
num_features = X.shape[1]
num_features

4

In [21]:
num_classes = Y.shape[1]
num_classes

3

In [22]:
'''
Y = W1.X1 + ... + W4.X4 + B
output = softmax(Y)
Create Weights (W) and Biases (B) Variables
'''

W = tf.Variable(tf.zeros([num_features, num_classes]))
B = tf.Variable(tf.zeros([num_classes]))

In [23]:
# Create x and y_ placeholders for holding actual data
x = tf.placeholder(tf.float32, [None, num_features])
y_ = tf.placeholder(tf.float32, [None, num_classes])

In [24]:
# Calculate y which holds the model's predicted logit values
Wx = tf.matmul(x, W)
y = Wx + B

In [25]:
# Create cost function to minimize
cost_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y))

In [26]:
# Create optimizer to minimize the cost function
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5).minimize(cost_cross_entropy)

In [29]:
def trainTheData(num_steps, optimizer_to_use, batch_size):
    init = tf.global_variables_initializer()
    # initialize all global variables
    
    # create a session
    with tf.Session() as sess:
        sess.run(init)
        
        for i in range(num_steps):
            
            # Calculate batch start index
            if batch_size == len(X_train):
                batch_start_index = 0
            elif batch_size > len(X_train):
                raise ValueError("Batch Size : " + str(batch_size) + " cannot be greater than Data Size : ", len(X_train))
            else:
                batch_start_index = (i * batch_size) % (len(X_train) - batch_size)
                
            # Calculate batch end index
            batch_end_index = batch_size + batch_start_index
            
            # Create Batch X and Y Values
            batch_X_values = X_train[batch_start_index : batch_end_index]
            batch_Y_values = Y_train[batch_start_index : batch_end_index]
            
            # Create feed dict to feed it to the optimizer
            feed = {x : np.array(batch_X_values), y_ : np.array(batch_Y_values)}
            
            sess.run(optimizer_to_use, feed_dict = feed)
            
            if (i + 1) % 50 == 0:
                print("After " + str(i + 1) + " iterations, cost : ", sess.run(cost_cross_entropy, feed_dict = feed))
                print("W : ", sess.run(W))
                print("B : ", sess.run(B))
                print("")
        
        # Store model's predicted logit values
        Y_pred_logits = sess.run(y, feed_dict = {x : np.array(X_test)})
        
        # Convert it to tensors
        Y_pred_logits_tensor = tf.convert_to_tensor(np.array(Y_pred_logits))
        
        # Apply softmax function
        apply_softmax = tf.nn.softmax(Y_pred_logits_tensor)
        
        # Store model's predicted classes
        Y_pred_classes = np.argmax(sess.run(apply_softmax), axis = 1)
            
        sess.close()
    return Y_pred_classes        

In [66]:
Y_pred_classes = trainTheData(num_steps = 200, optimizer_to_use = optimizer, batch_size = len(X_train))

After 50 iterations, cost :  0.236154
W :  [[-0.77554309  0.33894774  0.43659544]
 [ 0.94955462 -0.65572196 -0.29383257]
 [-1.21106172  0.0500053   1.1610564 ]
 [-1.13366997 -0.44651961  1.58018994]]
B :  [-0.4792183   1.10209072 -0.62287265]

After 100 iterations, cost :  0.166916
W :  [[-0.98325914  0.54376578  0.4394936 ]
 [ 1.21277785 -0.58903182 -0.62374628]
 [-1.57853818 -0.05927002  1.63780797]
 [-1.47997522 -0.72124046  2.20121503]]
B :  [-0.55887634  1.63064826 -1.07177222]

After 150 iterations, cost :  0.133598
W :  [[-1.12603462  0.66531616  0.46071887]
 [ 1.39155555 -0.5486865  -0.8428697 ]
 [-1.83635867 -0.15704125  1.99339998]
 [-1.72565067 -0.89455086  2.62020087]]
B :  [-0.58191395  1.99051607 -1.40860248]

After 200 iterations, cost :  0.113832
W :  [[-1.23471391  0.75358433  0.48112971]
 [ 1.52773297 -0.52445674 -1.00327682]
 [-2.03578758 -0.24512003  2.28090835]
 [-1.91680086 -1.0186553   2.93545604]]
B :  [-0.5871973   2.26398158 -1.67678428]



In [67]:
Y_test_classes = np.argmax(Y_test, axis = 1)

## Check the Accuracy

In [68]:
accuracy_score(Y_test_classes, Y_pred_classes)

0.93333333333333335

In [69]:
confusion_matrix(Y_test_classes, Y_pred_classes)

array([[16,  0,  0],
       [ 0,  4,  1],
       [ 0,  1,  8]], dtype=int64)